In [ ]:
%%time

from tqdm import tqdm

providers = ['WHISKEY', 'SIERRA']
start_date = 20241107
end_date = 20250107
date_range = integer_dates(start_date, end_date)

provider_dfs = {}

for provider in providers:
    dfs = []
    for day in tqdm(date_range, desc=f"{provider} dates"):
        query = f'''
            SELECT 
                S.CUEBIQ_ID, 
                H.EVENT_ZONED_DATETIME, 
                H.LNG, 
                H.LAT, 
                H.ACCURACY_METERS, 
                H.CLASSIFICATION_TYPE,
                Z.geography_id AS ZIPCODE_ID
            FROM {ping_table} H
            JOIN {out_usr_table} S 
                ON H.CUEBIQ_ID = S.CUEBIQ_ID
            JOIN {zip_codes_ca_name} Z
                ON ST_WITHIN(ST_MAKEPOINT(H.LNG, H.LAT), TO_GEOGRAPHY(Z.GEOG))
            WHERE H.PROCESSING_DATE = {day}
              AND H.COUNTRY_CODE = 'US'
              AND H.PROVIDER_ID = '{provider}'
        '''
        daily_df = snow_engine.read_sql(query)
        dfs.append(daily_df)
    provider_dfs[provider] = pd.concat(dfs, ignore_index=True)